In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [2]:
data = pd.read_csv('../Additional_eurecom_code/Event_extraction/data/joined_train.csv')

In [3]:
data = data.drop(data.columns[[0, 1]], axis=1)

In [ ]:
data = data[data['tag'].str.contains('effect')].reset_index(drop=True) #Only keep entries that have an effect and thus a relation
data.head(20)

In [23]:
"""
assumtpions:
- One subject and one object
- Every line has an effect
"""

data = data[data['tag'].str.contains('effect')] #Only keep entries that have an effect and thus a relation
triplets = []
labels = []
context = []
skipped = 0

for index, row in data.iterrows():

    sub_obj = {}

    tokenize_text = word_tokenize(row['text'])
    split_tags = row['tag'].split()

    if len(tokenize_text) != len(split_tags): #This means the tags and tokens do not overlap correctly, hence results are wrong.
        skipped += 1
        continue

    for word, tag in zip(tokenize_text, split_tags):

        if tag != '0': #an entity is detected

            if tag in sub_obj: #This means that the entity consists of 2 words
                sub_obj[tag] += f' {word}'
            else:
                sub_obj[tag] = word

    for key, value in sub_obj.items():
        if key != 'effect': #all other relations are the subjects
            subject = value
        else: #Effect is always the object
            object = value

    triplets.append(f"<triplet> {subject} <subj> {object} <obj> {row['label']}")
    labels.append(row['label'])
    context.append(row['text'])

final_data = pd.DataFrame(list(zip(range(len(triplets)), range(len(triplets)), context, triplets)),
                          columns = ['id', 'title', 'context', 'triplets'])

final_data.to_csv('Data/rebel_format.csv', index=False)
print(f"Number of skipped: {skipped}")

Number of skipped: 245


In [27]:
# to get some statistics
num_zero = 0
num_no_effect = 0
index = 0
for line, label in zip(data['tag'], data['label']):
    flag = False

    if label == '0':
        num_zero +=1
        continue

    for tag in line.split():
        if tag == 'effect':
            flag = True
            break

    if flag == False:
        num_no_effect +=1
        print(f"{index} no effect")

    index +=1
print(f"number of no relations: {num_zero}")
print(f"number of no effects: {num_no_effect}")

25 no effect
122 no effect
123 no effect
255 no effect
353 no effect
461 no effect
519 no effect
544 no effect
646 no effect
650 no effect
736 no effect
800 no effect
880 no effect
935 no effect
1023 no effect
1097 no effect
1203 no effect
1227 no effect
1272 no effect
1347 no effect
1387 no effect
1704 no effect
1785 no effect
1816 no effect
number of no relations: 96
number of no effects: 24


In [47]:
#Used for mapping the updated file where data is already split
data = pd.read_csv('Data/rebel/relation_data_updated.csv')
data= data.drop(data.columns[[0, 1, 2, 6, 8]], axis=1)
data = data[data.label != str(0)]
data = data.rename(columns={"sentence": "context"})
data['triplets'] = '<triplets> ' + data['trigger1'] + ' <subj> ' + data['trigger2'] + ' <obj> ' + data['label'] #Add the suitable format
data.to_csv('Data/rebel/rebel_format_v2.csv', index=False)